In [2]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all the individual amounts provided by the user.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```
Observation: 1280.26
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.2
  }
}
```
Observation: 2001.46
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```
Observation: 3941.75
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 3941.75,
    "b": 573.63
  }
}
```
Observation: 4515.38
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 4515.38,
    "b": 65.72
  }
}
```
Observation: 4581.1
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 4581.1,
    "b": 35.00
  }
}
```
Observation: 4616.1
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 

{'input': 'Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'I have calculated the total cost of all the individual amounts provided by the user. The final answer is $5273.38.'}

In [3]:
# SQLatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4-1106-preview",
)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``



Invoking: `sql_db_schema` with `movies, directors`
responded: The tables in the database are as follows:

- actors
- directors
- movies
- roles

To answer your question, I will need to look at the schema for the 'movies' and 'directors' tables to determine the relevant columns for movie budgets, votes, gross revenue, and director names. Let's check the schema for these tables.

Error: table_names {'directors', 'movies'} not found in database
Invoking: `sql_db_schema` with `actors, roles, movies`
responded: It seems there was an error with the table names 'movies' and 'directors'. They may not exist in the database or could be named differently. I will check the schema for the tables that do exist to find the correct table names and their columns. Let's check the schema for the 'actors', 'roles', and 'movies' tables, as these are the ones that could potentially contain the information we need.

Error: tab

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': 'I apologize for the confusion, but it seems there is an issue with retrieving the schema for the tables in the database. Without the correct table schema, I am unable to construct a query to answer your question about movies with the highest votes, lowest budgets, their directors, and gross revenue.\n\nIf you have any other questions or need assistance with something else, please let me know.'}